# Requirements

In [1]:
import cv2
import shutil
import random
import zipfile
import warnings
from PIL import Image
import numpy as np
# %load_ext cudf.pandas
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.resnet import ResNet50
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.metrics import precision_recall_fscore_support
from tensorflow.keras.callbacks import CSVLogger

# Suppress warnings
warnings.filterwarnings("ignore")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2024-06-16 19:27:06.511031: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 19:27:06.511131: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 19:27:06.647979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Balance the dataset by removing excess samples from the majority class
from random import sample
min_path = "/kaggle/input/skin-canser-b584m584/Melanoma-b584m584/malignant"
maj_path = "/kaggle/input/skin-canser-b584m584/Melanoma-b584m584/benign"
address = [image for image in os.listdir(maj_path)]
cut = len(os.listdir(min_path))
cut_list = sample(address, cut)
for index in tqdm(os.listdir(maj_path)):
    if index not in cut_list:
        os.remove(os.path.join(maj_path, index))

# Verify the number of samples
benigns = len(os.listdir("/kaggle/input/skin-canser-b584m584/Melanoma-b584m584/benign"))
melignant = len(os.listdir("/kaggle/input/skin-canser-b584m584/Melanoma-b584m584/malignant"))
print(f"\nNumber of benign Samples: {benigns}\nNumber of malignant Samples: {melignant}")

100%|██████████| 584/584 [00:00<00:00, 98701.44it/s]


Number of benign Samples: 584
Number of malignant Samples: 584


In [3]:
# Here since the path to dataset is read-only we need to copy them to another Dir
new_path = '/kaggle/working/root'
os.makedirs(new_path, exist_ok=True)
shutil.copytree('/kaggle/input/skin-canser-b584m584/Melanoma-b584m584', new_path, dirs_exist_ok=True)
# Just to Verify
print(os.listdir(new_path))

['b584m584.csv', 'malignant', 'benign']


In [4]:
# Function to divide a test set
def divide_test_set(temp_path, cut_percentage):
    """
    Returns a list containing relative address for images you need to move
    ------------------------------------------------------------------------
    temp_path: 
                    a path to the root directory of your data
    
    cut_precentage: 
                    how much data you want to move"""
    
    rel_image_paths = [os.path.join(temp_path, i) for i in os.listdir(temp_path)]
    cut_set = random.sample(rel_image_paths, int(cut_percentage * len(os.listdir(temp_path))))
    return cut_set

In [5]:
def copy_data(input_list, path):
    """Copies all the data located at the input list indexes
    -------------------------------------------------------------
    input_list: 
                a list containing all the relative paths
    
    path:
                output directory"""
    
    os.makedirs(path, exist_ok=True)
    for index in (input_list):
        shutil.copy(index, path) 

In [6]:
def move_data(input_list, path):
    """moves all the data located at the input list indexes
    -------------------------------------------------------------
    input_list: 
                a list containing all the relative paths
    
    path:
                output directory"""
    
    os.makedirs(path, exist_ok=True)
    for index in (input_list):
        shutil.move(index, path) 

In [7]:
def set_remainder(input_list, root_path):
    """
    This function removes the samples presented in input_list from the root path files
    -------------------------------------------------------------------------------------
    input_list: files to be excluded
    root_path: directory path
    
    """
    all_paths = [os.path.join(root_path, i) for i in os.listdir(root_path)]
    # creates a new list that only includes items from all_paths that are not in input_list
    filtered_list = [image for image in all_paths if image not in input_list]
    
    return filtered_list
    

In [8]:
# Data generator for training, validation and testing
def create_generator(DIR):
    datagen = ImageDataGenerator(rescale=1/255)
    generator = datagen.flow_from_directory(directory=DIR,
                                            batch_size=batch_size,
                                            class_mode='binary',
                                            target_size=(224, 224))
    return generator

In [9]:
def test_me(root):
    """
    This function captures prediction/label pairs and return predictions, labels lists
    ------------------------------------------------
    root: 
            a relative path to the test directory"""
    predictions = []
    labels = []
    for label in tqdm(os.listdir(root)):
        if label == "malignant":
            new_root = os.path.join(root, label)
            for image in tqdm(os.listdir(new_root)):
                # read, covert, and normalize the image
                img_path = os.path.join(new_root, image)
                image_file = Image.open(img_path).convert('RGB')
                image_array = np.array(image_file)
                # image_array = image_array * 255.0/image_array.max()
                image_array = cv2.resize(image_array, (224,224))
                image_array = image_array / 255.0
                image_array = image_array.reshape(1, 224,224, 3)
                # Prediction
                predictions.append(model.predict(image_array, verbose=0).squeeze())
                labels.append(1)
                
#             return predictions, labels 


        elif label == "benign":
            new_root = os.path.join(root, label)
            for image in tqdm(os.listdir(new_root)):
                # read, covert, and normalize the image
                img_path = os.path.join(new_root, image)
                image_file = Image.open(img_path).convert('RGB')
                image_array = np.array(image_file)
                # image_array = image_array * 255.0/image_array.max()
                image_array = cv2.resize(image_array, (224,224))
                image_array = image_array / 255.0
                image_array = image_array.reshape(1, 224,224, 3)
                # Prediction
                predictions.append(model.predict(image_array, verbose=0).squeeze())
                labels.append(0)
            
#             return predictions, labels 

        else:
            print("\nSomething is not right!")
    
    return predictions, labels

In [10]:
# Function to build the model
def build_model():
    image_size = 224
    ResNet50_base = ResNet50(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))
    model = ResNet50_base.output
    model = tf.keras.layers.GlobalAveragePooling2D()(model)
    model = tf.keras.layers.Dropout(rate=0.4)(model)
    model = tf.keras.layers.Dense(1, activation='sigmoid')(model)
    model = tf.keras.models.Model(inputs=ResNet50_base.input, outputs=model)
    return model

In [11]:
# Training parameters
batch_size = 8
EPOCHS = 40
test_accs = []
test_recalls = []
test_precisions = []
tps = []
fps = []
tns = []
fns = []


# Ensure TensorFlow uses the GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


for i in tqdm(range(6)):
    # TEST SAMPLING -----------------------------------------------
    # Pick a sample of images for testing 
    input_path = "/kaggle/working/root/benign"
    benign_test_set = divide_test_set(input_path, 0.20)
    input_path = "/kaggle/working/root/malignant"
    malignant_test_set = divide_test_set(input_path, 0.20)
    
    # Copy selected testing images into their corresponding folder
    copy_data(input_list = benign_test_set , path = f"/kaggle/working/test_{i+1}/benign")
    copy_data(input_list = malignant_test_set , path = f"/kaggle/working/test_{i+1}/malignant")
    
    # TRAIN SAMPLING ----------------------------------------------
    # Constant Paths
    root_benign_dir = "/kaggle/working/root/benign"
    root_malignant_dir = "/kaggle/working/root/malignant"
    
    # Get the training samples indexes
    benign_train_set = set_remainder(benign_test_set, root_benign_dir)
    malignant_train_set = set_remainder(malignant_test_set, root_malignant_dir)
    
    # Copy selected training images into their corresponding folder
    copy_data(input_list = benign_train_set , path = f"/kaggle/working/train_{i+1}/benign")
    copy_data(input_list = malignant_train_set , path = f"/kaggle/working/train_{i+1}/malignant")
    
    # VALIDATION SAMPLING -----------------------------------------
    # Pick a sample of images for validation 
    input_path = f"/kaggle/working/train_{i+1}/benign"
    benign_val_set = divide_test_set(input_path, 0.20)
    input_path = f"/kaggle/working/train_{i+1}/malignant"
    malignant_val_set = divide_test_set(input_path, 0.20)
    
    # Move selected validation images into their corresponding folder
    move_data(input_list = benign_val_set , path = f"/kaggle/working/val_{i+1}/benign")
    move_data(input_list = malignant_val_set , path = f"/kaggle/working/val_{i+1}/malignant")
    print(f"\n{i+1} out of 6 Dataset splitted.")
    
    # Used for Fit() function 
    va = len(os.listdir(f"/kaggle/working/val_{i+1}/benign")) # benign valiiation
    tr = len(os.listdir(f"/kaggle/working/train_{i+1}/benign")) # benign train
    va_ = len(os.listdir(f"/kaggle/working/val_{i+1}/malignant")) # malignant validation
    tr_ = len(os.listdir(f"/kaggle/working/train_{i+1}/malignant")) # malignant train
    print(f"\nNumber of benign train Samples: {tr}\nNumber of benign validation Samples: {va}")
    print(f"\nNumber of malignant train Samples: {tr_}\nNumber of malignant validation Samples: {va_}\n")
    
    # Create data generators
    Train_Dir = f"/kaggle/working/train_{i+1}/"
    train_generator = create_generator(DIR=Train_Dir)
    Val_Dir = f"/kaggle/working/val_{i+1}/"
    validation_generator = create_generator(DIR=Val_Dir)
    
    # Build a model and pick an optimizer
    model = build_model()
    opt = Adam(learning_rate=0.001)
    
    # Callback and Logger
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=4, min_delta=0.0001, mode='auto', verbose=1)
    csv_logger = CSVLogger(f'/kaggle/working/model_version_{i+1}_log.csv', append=True, separator=',')
    
    # Compile and Run
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy', 'Recall', 'Precision'])
    history = model.fit(
        train_generator,
        steps_per_epoch= (tr+tr_)//batch_size,
        epochs=EPOCHS,
        verbose=1,
        validation_data=validation_generator,
        callbacks=[reduce_lr, csv_logger]
    )
    
    model.save_weights(f'/kaggle/working/model_version_{i+1}.weights.h5')
    
    # Testing begins here ...
    test_path = f"/kaggle/working/test_{i+1}/"
    test_generator = create_generator(DIR=test_path)
    
    # Capture common metrics
    c_metrics = model.evaluate(test_generator)
    test_accs.append(c_metrics[1])
    test_recalls.append(c_metrics[2])
    test_precisions.append(c_metrics[3])
    
    # Make predictions
    predictions, labels = test_me(test_path)
    new_list = [0 if value <= 0.50 else 1 for value in predictions]
    print("-"*100)
    print(f"\npredictions before normalizing: {predictions}\n")
    print("-"*100)
    print(f"\npredictions after normalizing: {new_list}\n")
    print("-"*100)
    print(f"\nlabels: {labels}\n")
    print("="*100)
    
    cf = confusion_matrix(labels, new_list)
    tn = cf[0, 0]  # True Negatives
    fp = cf[0, 1]  # False Positives
    fn = cf[1, 0]  # False Negatives
    tp = cf[1, 1]  # True Positives

    # Just logging everything
    print(f"\nTraining number {i+1}/6 model performance:\nTN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}\n")
    tns.append(tn)
    fps.append(fp)
    fns.append(fn)
    tps.append(tp)
    print("="*100)
    
    # Clear the path for the next training index
    shutil.rmtree(f"/kaggle/working/train_{i+1}/")
    shutil.rmtree(f"/kaggle/working/val_{i+1}/")
    shutil.rmtree(f"/kaggle/working/test_{i+1}/")
    del model
    del history
    del train_generator, validation_generator, test_generator
    tf.keras.backend.clear_session()
    
    

  0%|          | 0/6 [00:00<?, ?it/s]


1 out of 6 Dataset splitted.

Number of benign train Samples: 375
Number of benign validation Samples: 93

Number of malignant train Samples: 375
Number of malignant validation Samples: 93

Found 750 images belonging to 2 classes.
Found 186 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/40
 1/93 ━━━━━━━━━━━━━━━━━━━━ 1:44:17 68s/step - Precision: 0.5714 - Recall: 1.0000 - accuracy: 0.6250 - loss: 0.8233

I0000 00:00:1718566117.382978     142 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


93/93 ━━━━━━━━━━━━━━━━━━━━ 144s 827ms/step - Precision: 0.6207 - Recall: 0.6328 - accuracy: 0.6186 - loss: 0.9424 - val_Precision: 0.5000 - val_Recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.6919 - learning_rate: 0.0010
Epoch 2/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 10s 112ms/step - Precision: 0.6000 - Recall: 0.7500 - accuracy: 0.6250 - loss: 0.5373 - val_Precision: 0.5000 - val_Recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.6916 - learning_rate: 0.0010
Epoch 3/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 41s 421ms/step - Precision: 0.6625 - Recall: 0.7109 - accuracy: 0.6658 - loss: 0.6713 - val_Precision: 0.5000 - val_Recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.6931 - learning_rate: 0.0010
Epoch 4/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 10s 111ms/step - Precision: 0.4286 - Recall: 1.0000 - accuracy: 0.5000 - loss: 0.6909 - val_Precision: 0.5000 - val_Recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.6931 - learning_rate: 0.0010
Epoch 5/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 45s 454ms/step - Precision: 0.64


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/116 [00:00<?, ?it/s]

  1%|          | 1/116 [00:04<08:58,  4.68s/it]

  2%|▏         | 2/116 [00:04<03:47,  2.00s/it]

  3%|▎         | 4/116 [00:05<01:31,  1.22it/s]

  5%|▌         | 6/116 [00:05<00:53,  2.05it/s]

  6%|▌         | 7/116 [00:05<00:44,  2.46it/s]

  7%|▋         | 8/116 [00:05<00:35,  3.05it/s]

  9%|▊         | 10/116 [00:05<00:24,  4.28it/s]

  9%|▉         | 11/116 [00:05<00:22,  4.57it/s]

 11%|█         | 13/116 [00:06<00:16,  6.10it/s]

 12%|█▏        | 14/116 [00:06<00:16,  6.20it/s]

 14%|█▍        | 16/116 [00:06<00:15,  6.56it/s]

 15%|█▍        | 17/116 [00:06<00:15,  6.38it/s]

 16%|█▋        | 19/116 [00:06<00:14,  6.53it/s]

 17%|█▋        | 20/116 [00:07<00:13,  6.87it/s]

 19%|█▉        | 22/116 [00:07<00:11,  7.83it/s]

 21%|██        | 24/116 [00:07<00:10,  9.04it/s]

 22%|██▏       | 25/116 [00:07<00:11,  7.82it/s]

 22%|██▏       | 26/116 [00:07<00:13,  6.64it/s]

 24%|██▍       | 28/116 [

...

 83%|████████▎ | 5/6 [1:41:44<20:31, 1231.47s/it]


6 out of 6 Dataset splitted.

Number of benign train Samples: 375
Number of benign validation Samples: 93

Number of malignant train Samples: 375
Number of malignant validation Samples: 93

Found 750 images belonging to 2 classes.
Found 186 images belonging to 2 classes.
Epoch 1/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 126s 734ms/step - Precision: 0.5544 - Recall: 0.5782 - accuracy: 0.5482 - loss: 1.1301 - val_Precision: 0.5000 - val_Recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 5.7844 - learning_rate: 0.0010
Epoch 2/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 10s 104ms/step - Precision: 0.6000 - Recall: 0.6000 - accuracy: 0.5000 - loss: 0.5207 - val_Precision: 0.5000 - val_Recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 9.9901 - learning_rate: 0.0010
Epoch 3/40
93/93 ━━━━━━━━━━━━━━━━━━━━ 48s 494ms/step - Precision: 0.5758 - Recall: 0.6349 - accuracy: 0.5822 - loss: 0.8232 - val_Precision: 0.5000 - val_Recall: 1.0000 - val_accuracy: 0.5000 - val_loss: 0.6931 - learning_rate: 0.0010
Epoch 4/40
93/93 ━━━━


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/116 [00:00<?, ?it/s]

  1%|          | 1/116 [00:03<06:45,  3.52s/it]

  3%|▎         | 3/116 [00:03<01:49,  1.03it/s]

  4%|▍         | 5/116 [00:03<00:56,  1.95it/s]

  6%|▌         | 7/116 [00:04<00:39,  2.76it/s]

  7%|▋         | 8/116 [00:04<00:33,  3.22it/s]

  8%|▊         | 9/116 [00:04<00:27,  3.86it/s]

  9%|▊         | 10/116 [00:04<00:24,  4.36it/s]

  9%|▉         | 11/116 [00:04<00:21,  4.85it/s]

 10%|█         | 12/116 [00:04<00:18,  5.63it/s]

 12%|█▏        | 14/116 [00:05<00:16,  6.09it/s]

 13%|█▎        | 15/116 [00:05<00:17,  5.93it/s]

 15%|█▍        | 17/116 [00:05<00:13,  7.24it/s]

 16%|█▌        | 18/116 [00:05<00:13,  7.36it/s]

 17%|█▋        | 20/116 [00:05<00:15,  6.31it/s]

 18%|█▊        | 21/116 [00:06<00:16,  5.92it/s]

 20%|█▉        | 23/116 [00:06<00:15,  5.90it/s]

 21%|██        | 24/116 [00:06<00:14,  6.32it/s]

 22%|██▏       | 25/116 [00:06<00:14,  6.46it/s]

 23%|██▎       | 27/116 [

----------------------------------------------------------------------------------------------------

predictions before normalizing: [array(0.5658067, dtype=float32), array(0.9938108, dtype=float32), array(0.83829564, dtype=float32), array(0.7836306, dtype=float32), array(0.41534087, dtype=float32), array(0.5947412, dtype=float32), array(0.5138341, dtype=float32), array(0.86924213, dtype=float32), array(0.93151736, dtype=float32), array(0.99155635, dtype=float32), array(0.6769271, dtype=float32), array(0.561859, dtype=float32), array(0.84952885, dtype=float32), array(0.92251945, dtype=float32), array(0.59552956, dtype=float32), array(0.4465179, dtype=float32), array(0.8315684, dtype=float32), array(0.76373464, dtype=float32), array(0.6335407, dtype=float32), array(0.09353875, dtype=float32), array(0.5251934, dtype=float32), array(0.9980124, dtype=float32), array(0.7462906, dtype=float32), array(0.80444795, dtype=float32), array(0.7569027, dtype=float32), array(0.676, dtype=float32), a

100%|██████████| 6/6 [2:02:27<00:00, 1224.60s/it]


In [ ]:
fprs = []

In [19]:
import plotly.graph_objects as go

# Calculate TPR (Recall) and FPR for each fold
TPR_list = [TP / (TP + FN) for TP, FN in zip(tps, fns)]
FPR_list = [FP / (FP + TN) for FP, TN in zip(fps, tns)]

# Create a figure
fig = go.Figure()

# Add ROC curves for each model
for i in range(len(TPR_list)):
    fig.add_trace(go.Scatter(x=[0, FPR_list[i], 1], y=[0, TPR_list[i], 1], mode='lines+markers', name=f'Model {i+1}'))

# Add the reference line
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'), name='Reference'))
fig.update_layout(
    title='Receiver Operating Characteristic (ROC) Curve',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    xaxis=dict(range=[0.0, 1.0]),
    yaxis=dict(range=[0.0, 1.05]),
    width=1100,
    height=400,
    legend=dict(x=0.8, y=0.1)
)

# Show the plot
fig.show()
